<a href="https://colab.research.google.com/github/jchen8000/MachineLearning/blob/master/10%20Generative%20Adversarial%20Network/Context-Conditional%20GAN/context_conditional_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Context-Conditional GAN

In [1]:
from keras.datasets import mnist
from keras.models import model_from_json
import numpy as np
import matplotlib.pyplot as plt
import scipy

Using TensorFlow backend.


In [2]:
!wget https://raw.githubusercontent.com/jchen8000/MachineLearning/master/10%20Generative%20Adversarial%20Network/Context-Conditional%20GAN/saved_model/ccgan_generator.json
!wget https://raw.githubusercontent.com/jchen8000/MachineLearning/master/10%20Generative%20Adversarial%20Network/Context-Conditional%20GAN/saved_model/ccgan_generator_weights.hdf5


--2019-09-08 00:11:59--  https://raw.githubusercontent.com/jchen8000/MachineLearning/master/10%20Generative%20Adversarial%20Network/Context-Conditional%20GAN/saved_model/ccgan_generator.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11746 (11K) [text/plain]
Saving to: ‘ccgan_generator.json’

ccgan_generator.jso 100%[===================>]  11.47K  --.-KB/s    in 0s      

2019-09-08 00:11:59 (144 MB/s) - ‘ccgan_generator.json’ saved [11746/11746]

--2019-09-08 00:12:00--  https://raw.githubusercontent.com/jchen8000/MachineLearning/master/10%20Generative%20Adversarial%20Network/Context-Conditional%20GAN/saved_model/ccgan_generator_weights.hdf5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ..

In [3]:
def load_model(model_name):
    model_path = "%s.json" % model_name
    weights_path = "%s_weights.hdf5" % model_name
    
    
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(weights_path)
    
    return loaded_model
  

generator = load_model("ccgan_generator")
#discriminator = load_model("ccgan_discriminator")


In [0]:
img_rows = 32
img_cols = 32
channels = 1
img_shape = (img_rows, img_cols, channels)
mask_height = 10
mask_width = 10
num_classes = 10
        
        
        
def mask_randomly(imgs):
    y1 = np.random.randint(0, img_rows - mask_height, imgs.shape[0])
    y2 = y1 + mask_height
    x1 = np.random.randint(0, img_rows - mask_width, imgs.shape[0])
    x2 = x1 + mask_width

    masked_imgs = np.empty_like(imgs)
    for i, img in enumerate(imgs):
        masked_img = img.copy()
        _y1, _y2, _x1, _x2 = y1[i], y2[i], x1[i], x2[i],
        masked_img[_y1:_y2, _x1:_x2, :] = 0
        masked_imgs[i] = masked_img
    return masked_imgs


def sample_images(imgs):
    r, c = 3, 6

    masked_imgs = mask_randomly(imgs)
    gen_imgs = generator.predict(masked_imgs)

    imgs = (imgs + 1.0) * 0.5
    masked_imgs = (masked_imgs + 1.0) * 0.5
    gen_imgs = (gen_imgs + 1.0) * 0.5

    gen_imgs = np.where(gen_imgs < 0, 0, gen_imgs)

    fig, axs = plt.subplots(r, c)
    for i in range(c):
        axs[0,i].imshow(imgs[i, :, :, 0], cmap='gray')
        axs[0,i].axis('off')
        axs[1,i].imshow(masked_imgs[i, :, :, 0], cmap='gray')
        axs[1,i].axis('off')
        axs[2,i].imshow(gen_imgs[i, :, :, 0], cmap='gray')
        axs[2,i].axis('off')
    plt.show()

In [0]:
import scipy.misc

# Load the dataset
(X_train, y_train), (_, _) = mnist.load_data()

# Rescale MNIST to 32x32
X_train = np.array([scipy.misc.imresize(x, [img_rows, img_cols]) for x in X_train])

# Rescale -1 to 1
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)
y_train = y_train.reshape(-1, 1)


idx = np.random.randint(0, X_train.shape[0], 6)
imgs = X_train[idx]